This exercise will require you to pull some data from the Quandl API. Quandl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Quandl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
START_DATE = '2020-03-09'
END_DATE = '2020-03-09'

request_params = { 'api_key': API_KEY, 'start_date': START_DATE, 'end_date': END_DATE }
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
r = requests.get(url, params=request_params)

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = json.loads(r.text)
print(json.dumps(json_data, indent=3))

{
   "dataset_data": {
      "limit": null,
      "transform": null,
      "column_index": null,
      "column_names": [
         "Date",
         "Open",
         "High",
         "Low",
         "Close",
         "Change",
         "Traded Volume",
         "Turnover",
         "Last Price of the Day",
         "Daily Traded Units",
         "Daily Turnover"
      ],
      "start_date": "2020-03-09",
      "end_date": "2020-03-09",
      "frequency": "daily",
      "data": [
         [
            "2020-03-09",
            87.65,
            92.15,
            85.1,
            87.2,
            null,
            392049.0,
            34832184.95,
            null,
            null,
            null
         ]
      ],
      "collapse": null,
      "order": null
   }
}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
START_DATE = '2017-01-01'
END_DATE = '2017-12-31'
DATE_INDEX = 0
OPEN_INDEX = 1
HIGH_INDEX = 2
LOW_INDEX = 3
CLOSE_INDEX = 4
TRADED_VOLUME_INDEX = 6

In [7]:
request_params = { 'api_key': API_KEY, 'start_date': START_DATE, 'end_date': END_DATE }
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
r = requests.get(url, params=request_params)

In [8]:
json_data = json.loads(r.text)
print(json_data['dataset_data']['data'][0])

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [9]:
open_prices = []
daily_highs = []
daily_lows = []
close_prices = []
traded_volumes = []

for daily_data in json_data['dataset_data']['data']:
    open_prices.append(daily_data[OPEN_INDEX])
    daily_highs.append(daily_data[HIGH_INDEX])
    daily_lows.append(daily_data[LOW_INDEX])
    close_prices.append(daily_data[CLOSE_INDEX])
    traded_volumes.append(daily_data[TRADED_VOLUME_INDEX])

print(open_prices[0])
print(daily_highs[0])
print(daily_lows[0])
print(close_prices[0])
print(traded_volumes[0])

51.76
51.94
51.45
51.76
34640.0


In [10]:
def average(values):
    val_count = len(values)
    val_sum = 0
    for v in values:
        val_sum += v
    return val_sum / val_count

In [11]:
def median(values):
    values.sort()
    val_count = len(values)
    if val_count % 2 == 0:
        val1 = values[val_count//2]
        val2 = values[(val_count//2) - 1]
        return (val1 + val2)/2
    else:
        return values[val_count//2]

In [12]:
# Some values for the opening price in the data were null.  This creates a problem for the sorting method and are irrelevant
# to the questions of highest and lowest opening prices, so they are removed first.
while None in open_prices:
    open_prices.remove(None)

sorted_open_prices = sorted(open_prices)
lowest_open = sorted_open_prices[0]
highest_open = sorted_open_prices[-1]
print('Lowest Open: ' + str(lowest_open))
print('Highest Open: ' + str(highest_open))

Lowest Open: 34.0
Highest Open: 53.11


In [13]:
greatest_daily_change = 0
for i in range(len(daily_highs)):
    daily_change = daily_highs[i] - daily_lows[i]
    if daily_change > greatest_daily_change:
        greatest_daily_change = daily_change
print('Greatest daily change in price: ' + str(greatest_daily_change))

Greatest daily change in price: 2.8100000000000023


In [14]:
greatest_close_price_change = 0
for i in range(len(close_prices) - 1):
    this_days_close = close_prices[0]
    next_days_close = close_prices[1]
    close_price_change = abs(this_days_close - next_days_close)
    if close_price_change > greatest_close_price_change:
        greatest_close_price_change = close_price_change
print('Greatest change in closing price between two days: ' + str(greatest_close_price_change))

Greatest change in closing price between two days: 0.1599999999999966


In [15]:
print('Average daily trading volume over year: ' + str(average(traded_volumes)))
print('Median trading volume over year: ' + str(median(traded_volumes)))

Average daily trading volume over year: 89124.33725490196
Median trading volume over year: 76286.0
